In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的AutoML表格分类模型与解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用 Python 的 Vertex 客户端库，使用 Google Cloud 的 [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 创建表格分类模型，并进行带有解释的在线预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。这个数据集不需要进行任何特征工程。您在本教程中将使用的数据集版本存储在公共云存储桶中。训练好的模型可以预测三种鸢尾花品种中的一种：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML表格分类模型，并部署进行在线预测以及可解释性。您也可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

所执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个服务端`Endpoint`资源。
- 使用可解释性进行预测。
- 撤消模型部署。

成本

本教程使用谷歌云（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您预期的使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您已经安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行本笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额，用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目ID。然后运行此单元格，以确保云SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 作为前缀的行作为 shell 命令，并将以 `$` 作为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于整个笔记本中的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持信息，请查看[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能会使用共享测试帐户或项目。为了避免用户在创建的资源之间发生命名冲突，您可以为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账号

**如果您正在使用谷歌云笔记本**，您的环境已经经过了身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并根据提示进行身份验证。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务账户密钥](https：//console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中，输入一个名称，然后点击**创建**。

在**将此服务账户授权访问项目**部分，点击Role下拉列表。在筛选框中输入“Vertex”，并选择**Vertex Administrator**。在筛选框中输入“Storage Object Admin”，并选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到本地环境。

在下面的单元格中，将您的服务账户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：数据集、模型、作业、流水线和端点服务的顶点API服务终点。
- `PARENT`：数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 自动机器学习常量

设置特定于AutoML数据集和训练的常量：

- 数据集模式：告诉“数据集”资源服务数据集的类型是什么。
- 数据标签(注释)模式：告诉“数据集”资源服务数据的标签(注释)是如何的。
- 数据集训练模式：告诉“流水线”资源服务要为哪种任务（例如分类）训练模型。

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

#### 硬件加速器

设置用于预测的硬件加速器（例如 GPU）。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个包含4个 Nvidia Telsa K80 GPU 的 GPU 容器映像分配给每个 VM，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 使用一个在 CPU 上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）映像

对于AutoML批量预测，服务二进制的容器映像是由Vertex预测服务预先确定的。更具体地说，该服务将根据您选择的硬件加速器来选择适当的模型容器。

#### 机器类型

接下来，设置机器类型用于预测。

- 设置变量 `DEPLOY_COMPUTE` 来配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 中的数字

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建您自己的AutoML表格分类模型。

## 设置客户端

Vertex 客户端库作为客户端/服务器模型运行。在您的一侧（Python脚本）上，您将创建一个客户端，该客户端将从 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。所以请提前设置好它们。

- 用于`Dataset`资源的数据集服务。
- 用于`Model`资源的模型服务。
- 用于训练的管道服务。
- 用于部署的端点服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，您的第一步是创建一个 `Dataset` 资源实例。这一步与 Vision、Video 和 Language 不同。对于这些产品，创建 `Dataset` 资源后，然后单独使用 `import_data` 方法导入数据。

对于表格数据，数据的导入将推迟到训练管道开始训练模型时。我们怎么做不同？首先，您不会调用 `import_data` 方法。相反，当您创建数据集实例时，您需要在 `Dataset` 资源的元数据中指定包含表格数据的 CSV 文件的 Cloud Storage 位置或数据表的 BigQuery 位置。

#### Cloud Storage

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Cloud Storage 路径的格式为:

    gs://[bucket_name]/[folder(s)/[file]

#### BigQuery

`metadata = {"input_config": {"bigquery_source": {"uri": [gcs_uri]}}}`

BigQuery 路径的格式为:

    bq://[collection].[dataset].[table]

请注意，`uri` 字段是一个列表，在您的数据跨文件分割时，您可以输入多个 CSV 文件或 BigQuery 表。

### 数据准备

Vertex` Dataset`资源对表格数据有一些要求。

- 必须是CSV文件或BigQuery查询。

CSV文件

对于表格分类，CSV文件有一些要求：

- 第一行必须是标题 -- 请注意，这与视觉、视频和语言不同，那里的要求是没有标题。
- 所有列除了一列都是特征。
- 一列是标签，在随后创建训练流程时您将指定。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的一个版本的Iris数据集，使用一个CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数 (`wc -l`) 来计算示例的数量，然后查看前几行。

您还需要知道标签列的标题名称，这个信息保存为`label_column`，对于这个数据集来说，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

数据集

现在您的客户已准备就绪，训练模型的第一步是创建一个受管数据集实例，然后将标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 创建 `Dataset` 资源的实例。此函数执行以下操作：

1. 使用数据集客户服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
- `display_name`：您选择的可读名称。
- `metadata_schema_uri`：数据集类型的模式。
- `metadata`：表格数据的 Cloud 存储或 BigQuery 位置。
3. 调用客户数据集服务方法 `create_dataset`，具有以下参数：
- `parent`：资源的 Vertex 位置根路径，用于您的 `Database`、`Model` 和 `Endpoint` 资源。
- `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用的方式。虽然这一步通常很快，但在项目中首次使用时，由于部署，会有较长延迟。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源）或取消操作，通过调用操作方法：

| 方法         | 描述                      |
| ------------ | ------------------------- |
| result()     | 等待操作完成并以 JSON 格式返回结果对象。    |
| running()    | 返回操作是否仍在运行的 True/False。      |
| done()       | 返回操作是否已完成的 True/False。   |
| canceled()   | 返回操作是否已取消的 True/False。   |
| cancel()     | 取消操作（可能需要多达 30 秒）。     |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("iris-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

## 训练模型

现在使用您的Vertex“数据集”资源训练一个AutoML表格分类模型。要训练模型，请按照以下步骤操作：

1. 为“数据集”资源创建一个Vertex训练管道。
2. 执行管道以开始训练。

### 创建训练流程

您可能会问，我们用管道做什么？通常在工作（例如训练）有多个步骤并且一般按顺序执行：执行步骤 A，执行步骤 B，等等时，您会使用管道。将这些步骤放入管道中，我们可以获得以下好处：

1. 可以用于后续的训练工作。
2. 可以容器化并作为批处理作业运行。
3. 可以分布式进行。
4. 所有步骤都与相同的管道工作相关联，以跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道工作的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 的完全限定数据集标识符。
- `schema`：数据集标注训练架构。
- `task`：描述训练工作要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练工作的完整规范。

现在让我们更深入地研究构建 `training_pipeline` 规范的 *最低* 要求：

- `display_name`：管道工作的人类可读名称。
- `training_task_definition`：数据集标注训练架构。
- `training_task_inputs`：描述训练工作要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
  - `dataset_id`：仅为 Vertex 数据集标识符（非完全限定）-- 这是完全限定标识符的最后一部分。
  - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与使用 Python（类似JSON）字典的其他参数不同，`task` 字段使用的是 Google protobuf Struct，这与 Python 字典非常相似。使用 `json_format.ParseDict` 方法进行转换。

您需要指定的最小字段包括：

- `prediction_type`：我们是在进行“分类”还是“回归”。
- `target_column`：我们想要预测的 CSV 表头列名（即标签）。
- `train_budget_milli_node_hours`：用于训练模型的最大预算时间，其中1000 = 1小时。
- `disable_early_stopping`：是True/False，用于让AutoML根据自己的判断是否提前停止训练或者训练到整个预算结束。
- `transformations`：为每个特征列指定特征工程。

对于 `transformations`，列表中必须包含每列一个条目。外键字段表示相应列的特征工程类型。在本教程中，将其设置为 `"auto"`，以告诉AutoML自动确定特征工程类型。

最后，通过调用助手函数 `create_pipeline` 创建管道来返回一个训练管道对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "sepal_width"}},
    {"auto": {"column_name": "sepal_length"}},
    {"auto": {"column_name": "petal_length"}},
    {"auto": {"column_name": "petal_width"}},
]

In [ ]:
PIPE_NAME = "iris_pipe-" + TIMESTAMP
MODEL_NAME = "iris_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "prediction_type": Value(string_value="classification"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "disable_early_stopping": Value(bool_value=False),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关培训管道的信息

现在仅获取此培训管道实例的管道信息。通过调用作业客户端服务的 `get_training_pipeline` 方法，利用以下参数的辅助函数获取仅此作业的作业信息：

- `name`：Vertex 完全合格的管道标识符。

当模型训练完成时，管道状态将为 `PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练以上模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，即流水线服务分配给它的标识符。您可以从返回的流水线实例中获取这个信息，这个字段是`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在你的模型已经训练完成，你可以获取一些关于你的模型的信息。

评估模型资源

现在找出模型服务认为您的模型有多好。在训练过程中，数据集的一部分被保留作为测试（留出）数据，由管道服务用于评估模型。

### 列出所有切片的评估

使用这个辅助函数 `list_model_evaluations`，它接受以下参数：

- `name`：`Model` 资源的 Vertex 完全限定模型标识符。

这个辅助函数使用模型客户端服务的 `list_model_evaluations` 方法，该方法接受相同的参数。调用返回的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），然后我们打印评估中每个指标的所有键名称，并对一小部分（`logLoss` 和 `auPrc`）打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

部署`Model`资源

现在部署使用AutoML创建的已训练的Vertex `Model`资源。 这需要两个步骤：

1. 为将`Model`资源部署到的`Endpoint`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个终端节点，用于部署模型以用于提供预测，包含以下参数：

- `display_name`: 用于 `Endpoint` 资源的可读性高的名称。

这个辅助函数使用 endpoint 客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: 用于 `Endpoint` 资源的可读性高的名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供预测的 `Endpoint` 资源做准备。您可以调用 `response.result()`，这是一个同步调用，并在终端节点准备就绪时返回。这个辅助函数返回 `Endpoint` 资源的 Vertex 完全合格的标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "iris_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的 `Endpoint` 资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例规模

您可以在处理在线预测请求时选择多种计算实例的规模：

- 单一实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为一。

- 手动调整：在线预测请求分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为相同数量的节点。当一个模型首次部署到实例上时，会预配固定数量的计算实例，并且在线预测请求会均匀分配到它们之间。

- 自动调整：在线预测请求会分配到可扩展数量的计算实例上。
  - 根据负载情况，将计算实例的最小（`MIN_NODES`）数量设定为在模型首次部署时需要预配和取消预配的数量，并将最大（`MAX_NODES`）数量设定为需要预配的计算实例的最大数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用此帮助函数 `deploy_model` 将模型部署到您为服务预测创建的端点，具有以下参数：

- `model`：要从训练管线上传（部署）的 `Model` 资源的 Vertex 完全限定标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。

帮助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`：部署模型的要求。
- `traffic_split`：发送到此模型的端点流量的百分比，以一个或多个键/值对的字典形式指定。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是要上传的此模型，100 表示 100% 的流量。
   - 如果端点上存在要分配流量的现有模型，则指定为如下所示，其中 `model_id` 是要部署到的现有模型的模型 ID。百分比必须相加为 100。

           { "0": 百分比, model_id: 百分比, ... }

现在让我们更深入地探讨 `deployed_model` 参数。此参数被指定为一个具有最少必填字段的 Python 字典：

- `model`：要部署的 (上传) `Model` 资源的 Vertex 完全限定标识符。
- `display_name`：部署模型的人类可读名称。
- `dedicated_resources`：指的是为服务预测请求扩展的计算实例（副本）的数量。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`：最初要提供的计算实例数量，您之前将其设置为变量 `MIN_NODES`。
  - `max_replica_count`：可扩展到的最大计算实例数量，您之前将其设置为变量 `MAX_NODES`。
- `enable_container_logging`：这会启用容器事件的日志记录，例如执行失败（默认情况下禁用容器日志记录）。容器日志记录通常在调试部署时启用，然后在生产部署时禁用。

#### 流量分割

现在让我们更深入地探讨 `traffic_split` 参数。此参数被指定为一个 Python 字典。可能一开始会有点困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，然后设置每个实例的百分比。

为什么要这样做？也许您已经在生产环境中部署了一个先前版本 -- 让我们称之为 v1。在 v2 上获得更好的模型评估，但在部署到生产环境之前，您不确定它是否真的更好。因此，在流量分割的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只分配到 10% 的流量。这样，您可以在不干扰大多数用户的情况下监控其表现，直到最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，这将阻塞直到模型部署完成。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的部署。

In [ ]:
DEPLOYED_NAME = "iris_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "enable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

使用可解释性进行在线预测请求

现在对已部署的模型进行在线预测，并使用可解释性。在这种方法中，预测的响应将包括有关特征如何对解释的贡献的解释。

制作测试项目

您将使用合成数据作为测试数据项。不要担心我们使用合成数据 - 我们只是想要展示如何进行预测。

In [ ]:
INSTANCE = {
    "petal_length": "1.4",
    "petal_width": "1.3",
    "sepal_length": "5.1",
    "sepal_width": "2.8",
}

### 进行预测并附带解释

好的，现在你有一个测试项目。使用这个辅助函数 `explain_item`，它接受以下参数：

- `data_items`: 测试表格数据项目。
- `endpoint`: 资源 `Model` 部署的 `Endpoint` 的 Vertex 完全限定标识符。
- `parameters_dict`: 用于提供预测结果的额外过滤参数 -- 在你的情况下，你将传递 `None`。

该函数使用预测客户端服务，并调用 `explain` 方法，带有以下参数：

- `endpoint`: 资源 `Model` 部署的 `Endpoint` 的 Vertex 完全限定标识符。
- `instances`: 要预测的实例（数据项目）列表。
- `parameters`: 用于提供预测结果的额外过滤参数。 *注意*，图像分割模型不支持额外参数。
- `deployed_model_id`: 部署模型的 Vertex 完全限定标识符，当端点上部署了多个模型时使用。否则，如果只部署了一个模型，可以设置为 `None`。

#### 请求

每个实例的格式为：

    { 'content': text_item }

由于 `explain()` 方法可以接受多个项目（实例），你将单个测试项目发送为一个测试项目列表。最后一步，你将实例列表打包成谷歌的协议缓冲区格式 -- 这是我们传递给 `explain()` 方法的内容。

#### 响应

`response` 对象返回一个列表，其中列表中的每个元素对应于请求中的相应图像。你将在每个预测的输出中看到：

- `deployed_model_id` -- 执行预测/解释的 `Model` 资源的 Vertex 完全限定标识符。
- `predictions` -- 预测类别和置信水平，介于 0 到 1 之间。
  - `confidences`: 预测的置信水平。
  - `displayNames`: 预测的标签。
- `explanations` -- 每个特征如何影响预测。

In [ ]:
def explain_item(
    data_items, endpoint, parameters_dict, deployed_model_id, silent=False
):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = [json_format.ParseDict(s, Value()) for s in data_items]

    response = clients["prediction"].explain(
        endpoint=endpoint,
        instances=instances,
        parameters=parameters,
        deployed_model_id=deployed_model_id,
    )
    if silent:
        return response

    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    try:
        predictions = response.predictions
        print("predictions")
        for prediction in predictions:
            print(" prediction:", dict(prediction))
    except:
        pass

    explanations = response.explanations
    print("explanations")
    for explanation in explanations:
        print(explanation)
    return response


response = explain_item([INSTANCE], endpoint_id, None, None)

理解解释回复

首先，您将查看您的模型预测并将其与实际值进行比较。

In [ ]:
import numpy as np

try:
    predictions = response.predictions
    label = np.argmax(predictions[0]["scores"])
    cls = predictions[0]["classes"][label]
    print("Predicted Value:", cls, predictions[0]["scores"][label])
except:
    pass

### 检查特征归因

接下来，您将查看此特定示例的特征归因。 正面归因值意味着特定特征通过该量推升了您模型的预测值，负面归因值则相反。

In [ ]:
from tabulate import tabulate

feature_names = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
attributions = response.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查您的解释和基线

为了更好地理解您得到的特征归因，您应该将它们与模型的基线进行比较。在大多数情况下，您的归因值之和加上基线应该与您模型对每个输入的预测值非常接近。还要注意，对于回归模型，从AI解释返回的 `baseline_score` 对于发送到您模型的每个示例将是相同的。对于分类模型，每个类别都有自己的基线。

在本节中，您将向模型发送 10 个测试示例以进行预测，以便将特征归因与基线进行比较。然后，您将通过 `sanity_check_explanations` 方法对每个测试示例的归因进行一次合理性检查。

#### 获取解释

In [ ]:
import random

# Prepare 10 test examples to your model for prediction using a random distribution to generate
# test instances
instances = []
for i in range(10):
    pl = str(random.uniform(1.0, 2.0))
    pw = str(random.uniform(1.0, 2.0))
    sl = str(random.uniform(4.0, 6.0))
    sw = str(random.uniform(2.0, 4.0))
    instances.append(
        {"petal_length": pl, "petal_width": pw, "sepal_length": sl, "sepal_width": sw}
    )

response = explain_item(instances, endpoint_id, None, None, silent=True)

在下面的函数中，您对说明进行了一次健全性检查。

In [ ]:
def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

## 取消部署 `Model` 资源

现在从提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到终端点服务时返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

这个函数调用了端点客户服务的 `undeploy_model` 方法，带有以下参数：

- `deployed_model_id`：在将 `Model` 资源部署时终端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上的剩余部署模型之间分配流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空对象 {} 来简单地将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME